In [1]:
import os
import re
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
col_Names =["userID", "gender", "age", "occupation", "starSign", "date", "text"]




In [2]:
df_train = pd.read_csv('train_raw.csv', names=col_Names)
print("read train_raw.csv succesfully")


read train_raw.csv succesfully


In [3]:
df_test = pd.read_csv('dev_raw.csv',names=col_Names)
print("read t_raw.csv succesfully")
df_test = (df_test.groupby(['userID','age'])['text'].apply(' '.join).reset_index())
df_test.loc[~df_test.age.isin([14,15,16,24,25,26,34,35,36,44,45,46]),'age'] = 0
df_test.loc[df_test.age.isin([14,15,16]),'age'] = 1
df_test.loc[df_test.age.isin([24,25,26]),'age'] = 2
df_test.loc[df_test.age.isin([34,35,36]),'age'] = 3
df_test.loc[df_test.age.isin([44,45,46]),'age'] = 4


read t_raw.csv succesfully


In [4]:
#remove empty space and start spaces from normalise train dataFrame
indices_to_remove=[]
index=0
for row in df_train.itertuples():
    text = str(row.text.strip())
    df_train.at[row[0],'text']=text
    if len(text)==0:
        indices_to_remove.append(index)
    index+=1
df_train=df_train.drop(df_train.index[indices_to_remove])
df_train=df_train.reset_index()

In [5]:
df_train = (df_train.groupby(['userID','age'])['text'].apply(' '.join).reset_index())


In [6]:
df_train.loc[df_train.age.isin([14,15,16]),'age'] = 1
df_train.loc[df_train.age.isin([24,25,26]),'age'] = 2
df_train.loc[df_train.age.isin([34,35,36]),'age'] = 3
df_train.loc[df_train.age.isin([44,45,46]),'age'] = 4


In [7]:
df_test_check = pd.read_csv('dev_raw.csv',names=col_Names)
print("read dev_raw.csv succesfully")
df_test_check.loc[df_test_check.age.isin([14,15,16]),'age'] = 1
df_test_check.loc[df_test_check.age.isin([24,25,26]),'age'] = 2
df_test_check.loc[df_test_check.age.isin([34,35,36]),'age'] = 3
df_test_check.loc[df_test_check.age.isin([44,45,46]),'age'] = 4

read dev_raw.csv succesfully


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
y=df_train['age']
smooth=[0.000001,0.0001,0.001,0.01,1]
for a in smooth:
    text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', MultinomialNB(alpha=a)),])
    text_clf = text_clf.fit(df_train['text'], y)
    predicted = text_clf.predict(df_test['text'])
    userID_list=list(df_test['userID'])
    
    from collections import defaultdict

    dict_predict=defaultdict(int)
    for i in range(0,len(predicted)):
        dict_predict[userID_list[i]]=predicted[i]

    correct_result=[]
    for row in df_test_check.itertuples():
        correct_result.append((df_test_check.at[row[0],'userID'],df_test_check.at[row[0],'age']))
    predict=0
    total = -1
    not_p = -1
    for i in range(0,len(correct_result)):
        total+=1
        userID_to_check=correct_result[i][0]
        if dict_predict[userID_to_check] == correct_result[i][1]:
            predict+=1
        else:
            not_p+=1

    print("accuracy for naive baeyes is ",predict/len(correct_result))
    target_names=["0","1","2","3","4"]
    report_nb=classification_report(df_test['age'],predicted,target_names=target_names)
    y_p_nb=[]
    for i in range(0,len(correct_result)):
        y_p_nb.append(dict_predict[correct_result[i][0]] )
    y_t_nb=[x[1] for x in correct_result]
    target_names=["14-16","24-26","34-36","44-46"]
    report_nb=classification_report(y_t_nb,y_p_nb,target_names=target_names)
    print(report_nb)




accuracy for naive baeyes is  0.5914144533662755
             precision    recall  f1-score   support

      14-16       0.72      0.80      0.76     13100
      24-26       0.53      0.94      0.68     17298
      34-36       0.06      0.01      0.01      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.42      0.59      0.48     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for naive baeyes is  0.5818185828994971
             precision    recall  f1-score   support

      14-16       0.70      0.81      0.75     13100
      24-26       0.52      0.89      0.66     17298
      34-36       0.37      0.10      0.16      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.42      0.58      0.48     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for naive baeyes is  0.5738551133856878
             precision    recall  f1-score   support

      14-16       0.70      0.83      0.76     13100
      24-26       0.52      0.86      0.65     17298
      34-36       0.21      0.10      0.14      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.41      0.57      0.48     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for naive baeyes is  0.581972999205859
             precision    recall  f1-score   support

      14-16       0.70      0.83      0.76     13100
      24-26       0.52      0.90      0.66     17298
      34-36       0.19      0.00      0.00      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.41      0.58      0.47     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for naive baeyes is  0.5818185828994971
             precision    recall  f1-score   support

      14-16       0.69      0.82      0.75     13100
      24-26       0.52      0.90      0.66     17298
      34-36       0.00      0.00      0.00      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.40      0.58      0.47     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


In [14]:
y_p_nb=[]
for i in range(0,len(correct_result)):
    y_p_nb.append(dict_predict[correct_result[i][0]] )
y_t_nb=[x[1] for x in correct_result]
target_names=["14-16","24-26","34-36","44-46"]
report_nb=classification_report(y_t_nb,y_p_nb,target_names=target_names)

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
new_pred_nb=[]
for i in range(0,len(correct_result)):
    (correct_result[i][0],dict_predict[correct_result[i][0]])
    if dict_predict[correct_result[i][0]] ==1:
        new_pred_nb.append((correct_result[i][0],"14-16"))
    if dict_predict[correct_result[i][0]] ==2:
        new_pred_nb.append((correct_result[i][0],"24-26"))
    if dict_predict[correct_result[i][0]]==3:
        new_pred_nb.append((correct_result[i][0],"34-36"))
    if dict_predict[correct_result[i][0]] ==4:
        new_pred_nb.append((correct_result[i][0],"44-46"))

In [16]:
from collections import defaultdict


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from collections import defaultdict
C=[1,3,5,10,13,20,100]
# C=[20]

y=df_train['age']
for al in C:
    text_clf_lg = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                                  ('clf-svm',LogisticRegression(C=al)),])

    _ = text_clf_lg.fit(df_train['text'], y)
    predicted_lg = text_clf_lg.predict(df_test['text'])
    userID_list=list(df_test['userID'])


    dict_predict=defaultdict(int)
    for i in range(0,len(predicted_lg)):
        dict_predict[userID_list[i]]=predicted_lg[i]

    correct_result=[]
    for row in df_test_check.itertuples():
        correct_result.append((df_test_check.at[row[0],'userID'],df_test_check.at[row[0],'age']))
    predict=0
    total = -1
    not_p = -1
    for i in range(0,len(correct_result)):
        total+=1
        userID_to_check=correct_result[i][0]
        if dict_predict[userID_to_check] == correct_result[i][1]:
            predict+=1
        else:
            not_p+=1
    print("accuracy for logistic regression with alpha = ",al , " ", predict/len(correct_result))
    target_names=["0","1","2","3","4"]
    report_lg=classification_report(df_test['age'],predicted_lg,target_names=target_names)
    y_p_lg=[]
    for i in range(0,len(correct_result)):
        y_p_lg.append(dict_predict[correct_result[i][0]] )
    y_t_lg=[x[1] for x in correct_result]
    target_names=["14-16","24-26","34-36","44-46"]
    report_lg=classification_report(y_t_lg,y_p_lg,target_names=target_names)
    print(report_lg)
    


    


        

accuracy for logistic regression with alpha =  1   0.6096355775169858
             precision    recall  f1-score   support

      14-16       0.69      0.90      0.78     13100
      24-26       0.56      0.91      0.70     17298
      34-36       0.00      0.00      0.00      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.41      0.61      0.49     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for logistic regression with alpha =  3   0.6306803141268861
             precision    recall  f1-score   support

      14-16       0.72      0.93      0.81     13100
      24-26       0.58      0.95      0.72     17298
      34-36       0.36      0.01      0.01      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.45      0.63      0.51     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for logistic regression with alpha =  5   0.6321582987734934
             precision    recall  f1-score   support

      14-16       0.70      0.94      0.80     13100
      24-26       0.59      0.94      0.73     17298
      34-36       0.15      0.01      0.02      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.44      0.63      0.51     45332



...

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for logistic regression with alpha =  20   0.6404747198447013
             precision    recall  f1-score   support

      14-16       0.69      0.95      0.80     13100
      24-26       0.61      0.95      0.75     17298
      34-36       0.32      0.09      0.13      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.45      0.64      0.52     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for logistic regression with alpha =  100   0.6354010412070943
             precision    recall  f1-score   support

      14-16       0.71      0.95      0.81     13100
      24-26       0.63      0.93      0.75     17298
      34-36       0.13      0.10      0.11      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.45      0.64      0.53     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


In [67]:
#played with confidence of best model of class preiction
# proba_lg = text_clf_lg.predict_proba(df_test['text'])
# class1_1=[]
# class1_not_1=[]
# class2_2=[]
# class2_not_2=[]
# class1_1_2=[]
# i=-1
# pr=0
# ac=0
# predicted_lg = text_clf_lg.predict(df_test['text'])
# for row in df_test.itertuples():
#     i+=1
#     if df_test.loc[i]['age'] == predicted_lg[i] == 1:
#         class1_1.append(proba_lg[i][0])
# #         print(proba_lg[i])
# #         print(proba_lg[i][0]-proba_lg[i][1])
#         class1_1_2.append(proba_lg[i][0]-proba_lg[i][1])
#     if df_test.loc[i]['age'] not in [11,12,13,14,15,16,17,18,19] and predicted_lg[i] ==1:
#         class1_not_1.append(proba_lg[i][0])
#     if df_test.loc[i]['age'] == predicted_lg[i] == 2:
#         class2_2.append(proba_lg[i][1])
#     if df_test.loc[i]['age'] not in  [21,22,23,24,25,26,27,28,29] and predicted_lg[i] ==2:
#         class2_not_2.append(proba_lg[i][1])
    
# #     if df_test.loc[i]['age'] in [11,12,13,14,15,16,17,18,19]  and predicted_lg[i] !=1:
# #         1==1
#     if  proba_lg[i][0] in (0.23,0.24,0.25,0.26,.27,.28) and proba_lg[i][1] in (0.23,0.24,0.25,0.26,.27,.28):
#           predicted_lg[i]=0
        
# i=-1
# pr=0
# ac=0
# for row in df_test.itertuples():
#     i+=1
#     if abs(proba_lg[i][0]-proba_lg[i][1]) < 0.09  and proba_lg[i][0]<0.5 and proba_lg[i][1]<0.5:
#           predicted_lg[i]=0
        
# #     if /abs(proba_lg[i][0]-proba_lg[i][1]) < 0.03  and proba_lg[i][0]<0.6 and proba_lg[i][1]<0.6:
   
# print(np.mean(predicted_lg==df_test['age']))

# userID_list=list(df_test['userID'])


# dict_predict=defaultdict(int)
# for i in range(0,len(predicted_lg)):
#     dict_predict[userID_list[i]]=predicted_lg[i]

# correct_result=[]
# for row in df_test_check.itertuples():
#     correct_result.append((df_test_check.at[row[0],'userID'],df_test_check.at[row[0],'age']))
# predict=0
# total = -1
# not_p = -1
# for i in range(0,len(correct_result)):
#     total+=1
#     userID_to_check=correct_result[i][0]
#     if dict_predict[userID_to_check] == correct_result[i][1]:
#         predict+=1
#     else:
#         not_p+=1

# print(predict/len(correct_result))
# print()  
   
    

read dev_raw.csv succesfully


In [18]:
from sklearn.linear_model import SGDClassifier
alphas = [1e-5,1e-3,1e-1,1]
for a in alphas:
    text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                             ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=a, max_iter=5, random_state=42)),])
    _ = text_clf_svm.fit(df_train['text'], y)
    predicted_svm = text_clf_svm.predict(df_test['text'])
    userID_list=list(df_test['userID'])


    dict_predict=defaultdict(int)
    for i in range(0,len(predicted)):
        dict_predict[userID_list[i]]=predicted_svm[i]

    correct_result=[]
    for row in df_test_check.itertuples():
        correct_result.append((df_test_check.at[row[0],'userID'],df_test_check.at[row[0],'age']))
    predict=0
    total = -1
    not_p = -1
    for i in range(0,len(correct_result)):
        total+=1
        userID_to_check=correct_result[i][0]
        if dict_predict[userID_to_check] == correct_result[i][1]:
            predict+=1
        else:
            not_p+=1

    print("accuracy for SVM classifier is as :",predict/len(correct_result))
    target_names=["0","1","2","3","4"]
    report_svm=classification_report(df_test['age'],predicted_svm,target_names=target_names)
    y_p_svm=[]
    for i in range(0,len(correct_result)):
        y_p_svm.append(dict_predict[correct_result[i][0]] )
    y_t_svm=[x[1] for x in correct_result]

    target_names=["14-16","24-26","34-36","44-46"]
    report_svm=classification_report(y_t_svm,y_p_svm,target_names=target_names)
    print(report_svm)
 




accuracy for SVM classifier is as : 0.62816553428042
             precision    recall  f1-score   support

      14-16       0.66      0.94      0.78     13100
      24-26       0.61      0.92      0.73     17298
      34-36       0.27      0.05      0.08      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.44      0.63      0.51     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for SVM classifier is as : 0.584068649077914
             precision    recall  f1-score   support

      14-16       0.67      0.84      0.74     13100
      24-26       0.54      0.89      0.67     17298
      34-36       0.00      0.00      0.00      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.40      0.58      0.47     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for SVM classifier is as : 0.4154681020030001
             precision    recall  f1-score   support

      14-16       0.83      0.12      0.21     13100
      24-26       0.40      1.00      0.57     17298
      34-36       0.00      0.00      0.00      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.39      0.42      0.28     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


accuracy for SVM classifier is as : 0.41676961087090797
             precision    recall  f1-score   support

      14-16       0.82      0.12      0.21     13100
      24-26       0.40      1.00      0.57     17298
      34-36       0.00      0.00      0.00      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.39      0.42      0.28     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))


In [86]:

#decision Tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn as sns
# %matplotlib inlin
from sklearn.metrics import accuracy_score
#best split at 1700

text_clf_tree = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('c',DecisionTreeClassifier(min_samples_split =1700)),])

dt = text_clf_tree.fit(df_train['text'], y)
predicted_tree = text_clf_tree.predict(df_test['text'])
np.mean(predicted_tree == df_test['age'])
userID_list=list(df_test['userID'])


dict_predict=defaultdict(int)
for i in range(0,len(predicted)):
    dict_predict[userID_list[i]]=predicted_tree[i]

correct_result=[]
for row in df_test_check.itertuples():
    correct_result.append((df_test_check.at[row[0],'userID'],df_test_check.at[row[0],'age']))
predict=0
total = -1
not_p = -1
for i in range(0,len(correct_result)):
    total+=1
    userID_to_check=correct_result[i][0]
    if dict_predict[userID_to_check] == correct_result[i][1]:
        predict+=1
    else:
        not_p+=1

print("decision tree predicted the accuracy of  ", predict/len(correct_result))

new_pred_tree=[]
for i in range(0,len(correct_result)):
    (correct_result[i][0],dict_predict[correct_result[i][0]])
    if dict_predict[correct_result[i][0]] ==1:
        new_pred_tree.append((correct_result[i][0],"14-16"))
    if dict_predict[correct_result[i][0]] ==2:
        new_pred_tree.append((correct_result[i][0],"24-26"))
    if dict_predict[correct_result[i][0]]==3:
        new_pred_tree.append((correct_result[i][0],"34-36"))
    if dict_predict[correct_result[i][0]] ==4:
        new_pred_tree.append((correct_result[i][0],"44-46"))

y_p_tree=[]
for i in range(0,len(correct_result)):
    y_p_tree.append(dict_predict[correct_result[i][0]] )
y_t_tree=[x[1] for x in correct_result]
target_names=["14-16","24-26","34-36","44-46"]
report_tree=classification_report(y_t_tree,y_p_tree,target_names=target_names)
print(report_tree)

0.5198535251036795
45332
45332
             precision    recall  f1-score   support

      14-16       0.51      0.70      0.59     13100
      24-26       0.53      0.83      0.64     17298
      34-36       0.00      0.00      0.00      2584
      44-46       0.00      0.00      0.00       551

avg / total       0.35      0.52      0.42     45332



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 18, does not match size of target_names, 4
  .format(len(labels), len(target_names))
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
